In [1]:
from pulp import *

In [ ]:
# stanoveni primalu

suppliers = ['D1', 'D2', 'D3']             # množina I
customers = ['O1', 'O2', 'O3', 'O4']       # množina J

capacity = {                               #  s_i   (pravé strany kapacit)
    'D1': 20,
    'D2': 30,
    'D3': 25
}

demand = {                                 #  d_j   (rovnostní pravé strany poptávky)
    'O1': 10,
    'O2': 25,
    'O3': 25,
    'O4': 15
}

# koeficiekty promennych
cost = {
    ('D1','O1'): 2,  ('D1','O2'): 3,  ('D1','O3'): 1,  ('D1','O4'): 4,
    ('D2','O1'): 3,  ('D2','O2'): 2,  ('D2','O3'): 4,  ('D2','O4'): 1,
    ('D3','O1'): 5,  ('D3','O2'): 6,  ('D3','O3'): 2,  ('D3','O4'): 3
}

#    O1  O2  O3  O4
# D1  2   3   1   4 20
# D2  3   2   4   1 30
# D3  5   6   2   3 25
#    10   25  25  15

In [3]:
# příprava modelu
model = LpProblem("Transport", LpMinimize)
x = LpVariable.dicts("x",
                     (suppliers, customers),
                     lowBound=0)

In [4]:
# objective function: cílová funkce (minimalizace nákladů)

model += lpSum(cost[i,j] * x[i][j]
               for i in suppliers
               for j in customers), "TotalCost"


minimalizuj = 2 x<sub>11</sub> + 3 x<sub>12</sub> + x<sub>13</sub> + 4 x<sub>14</sub> + 3 x<sub>21</sub> + 2 x<sub>22</sub> + 4 x<sub>23</sub> + x<sub>24</sub> + 5 x<sub>31</sub> + 6 x<sub>32</sub> + 2 x<sub>33</sub> + 3 x<sub>34</sub>

In [5]:
# constraints: omezení proměnných

# Dodavatelé – kapacitní nerovnosti   Σ_j x_{ij} ≤ s_i
for i in suppliers:
    model += lpSum(x[i][j] for j in customers) <= capacity[i], f"supply_{i}"

# Odběratelé – poptávka   Σ_i x_{ij} = d_j
for j in customers:
    model += lpSum(x[i][j] for i in suppliers) == demand[j], f"demand_{j}"

D1: x<sub>11</sub> + x<sub>12</sub> + x<sub>13</sub> + x<sub>14</sub> <= 20  
D2: x<sub>21</sub> + x<sub>22</sub> + x<sub>23</sub> + x<sub>24</sub> <= 30  
D3: x<sub>31</sub> + x<sub>32</sub> + x<sub>33</sub> + x<sub>34</sub> <= 30  
  
O1: x<sub>11</sub> + x<sub>21</sub> + x<sub>31</sub> = 10  
O1: x<sub>12</sub> + x<sub>22</sub> + x<sub>32</sub> = 25  
O1: x<sub>13</sub> + x<sub>23</sub> + x<sub>33</sub> = 25  
O1: x<sub>14</sub> + x<sub>24</sub> + x<sub>34</sub> = 15  

x<sub>ij</sub> >= 0

In [6]:
# řešení

model.solve()

1

In [7]:
# řešení primalu

print(f"\n✅  Stav: {LpStatus[model.status]}")
print(f"💰  Minimální náklady: {value(model.objective):.2f} Kč\n")

print("📦  Přepravní plán (x_{ij} > 0):")
for i in suppliers:
    for j in customers:
        q = x[i][j].varValue
        if q > 1e-6:
            print(f"   {i} → {j}: {q:.1f} jednotek")


✅  Stav: Optimal
💰  Minimální náklady: 145.00 Kč

📦  Přepravní plán (x_{ij} > 0):
   D1 → O1: 10.0 jednotek
   D1 → O3: 10.0 jednotek
   D2 → O2: 25.0 jednotek
   D2 → O4: 5.0 jednotek
   D3 → O3: 15.0 jednotek
   D3 → O4: 10.0 jednotek


In [8]:
# slacky (fiktivní dodavatelé) a stínové ceny (váhy)
print("\n📊  Slacky a duální proměnné")
print("-------------------------------------------------------------")
print(f"{'Omezení':<12} | {'Slack':>7} | {'Dual':>9} | Aktivní")
print("-------------------------------------------------------------")

for c in model.constraints.values():
    name  = c.name
    slack = c.slack                 # b − A·x
    dual  = c.pi                    # stínová cena
    active = abs(slack) < 1e-6
    print(f"{name:<12} | {slack:7.1f} | {dual:9.2f} | {active}")


📊  Slacky a duální proměnné
-------------------------------------------------------------
Omezení      |   Slack |      Dual | Aktivní
-------------------------------------------------------------
supply_D1    |    -0.0 |     -1.00 | True
supply_D2    |    -0.0 |     -2.00 | True
supply_D3    |    -0.0 |      0.00 | True
demand_O1    |    -0.0 |      3.00 | True
demand_O2    |    -0.0 |      4.00 | True
demand_O3    |    -0.0 |      2.00 | True
demand_O4    |    -0.0 |      3.00 | True


In [9]:
# kontrola duality (optimalizace)

dual_value = (
    sum(capacity[i] * model.constraints[f"supply_{i}"].pi for i in suppliers) +
    sum(demand[j]   * model.constraints[f"demand_{j}"].pi for j in customers)
)
gap = value(model.objective) - dual_value
print(f"\n🔍  Primal = {value(model.objective):.2f},  Dual = {dual_value:.2f},  Gap = {gap:.2e}")


🔍  Primal = 145.00,  Dual = 145.00,  Gap = 0.00e+00
